<a href="https://colab.research.google.com/github/axeltanjung/qa_rag_public_policy/blob/main/src/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unlocking Solutions: Addressing Fragmented Public Policy and Accountability Challenges in Decentralized Indonesia through Langchain and FAISS-driven Retrieve-and-Generate Approach


## 1.	Introduction & Background

Indonesia's transition towards decentralized governance marks a significant shift in its political landscape, aiming to empower local authorities and enhance public service delivery across its diverse regions. However, amidst the benefits of decentralization, challenges have emerged, particularly concerning fragmented public policy and accountability mechanisms. These challenges pose significant obstacles to effective governance and sustainable development. In response to these pressing issues, innovative technologies such as Langchain and FAISS have garnered attention for their potential to revolutionize policymaking processes. Leveraging advanced natural language processing and information retrieval techniques, these technologies offer a unique approach to navigating complex policy landscapes and generating actionable insights.

Decentralization in Indonesia, initiated in the late 1990s, aimed to democratize governance and promote local autonomy. As a result, provincial and district governments gained substantial decision-making power in various sectors. However, this devolution of authority has led to fragmented policymaking processes, characterized by inconsistencies and inefficiencies across different regions. Moreover, the decentralized governance structure has exposed gaps in accountability mechanisms, raising concerns about transparency and the effective allocation of resources. Corruption and mismanagement remain persistent challenges, hindering the equitable distribution of public services and impeding development efforts.
Recognizing the need for innovative solutions, there is growing interest in harnessing technology to address these governance challenges. Langchain and FAISS, with their advanced capabilities in natural language processing and information retrieval, offer promising avenues for enhancing policy coherence and accountability. Fragmented public policy arises from the diverse decision-making authorities across Indonesia's decentralized governance structure. Each province and district has the autonomy to formulate policies tailored to its specific needs and priorities. While this decentralization fosters local empowerment, it also creates challenges in aligning policies at the national level and coordinating responses to shared issues such as healthcare, education, and environmental conservation.

Moreover, the lack of robust accountability mechanisms exacerbates these challenges. In many cases, local governments face limited oversight, allowing for mismanagement and corruption to thrive. Without transparent and accountable governance processes, there is a risk of resources being misallocated or diverted, undermining the effectiveness of public service delivery and eroding public trust in government institutions. In this context, Langchain and FAISS offer innovative solutions by providing policymakers with tools to navigate the complexities of decentralized governance effectively. Langchain's natural language processing capabilities enable the analysis of vast amounts of textual data, including policy documents, legislative texts, and public feedback. This facilitates the identification of policy gaps, inconsistencies, and areas for improvement.

Additionally, FAISS's efficient information retrieval algorithms enable policymakers to access relevant information quickly and accurately. By aggregating data from various sources, including academic research, government reports, and media coverage, FAISS empowers policymakers to make evidence-based decisions and formulate informed policies. Through the integration of Langchain and FAISS into policymaking processes, Indonesia can overcome the challenges of fragmented public policy and accountability in decentralized governance. By promoting policy coherence, transparency, and data-driven decision-making, these technologies pave the way for more effective and inclusive governance practices, ultimately contributing to the country's development goals.


## 2.	Dataset

•	**Title**: Serving The Political Parties: Issues Of Fragmented Public Policy And Accountability In Decentralized Indonesia

•	**Author**: Wahyudi Kumorotomo

•	**Affiliation**: Department of Public Administration, Gadjah Mada University, Indonesia

•	**Presented at**: Fourth International Conference on Public Policy and Management (CPPM), Indian Institute of Management, Bangalore, India (August 9-12, 2009) 

Source of original dataset can be access through this link:

http://www.kumoro.staff.ugm.ac.id/file_artikel/Full%20paper-Serving%20the%20Political%20Parties.pdf


## 3.	Project Development

### Task 1 : Adding Document

**Install All Dependencies**

In [68]:
!pip install langchain
!pip install unstructured
!pip install pdf2image
!pip install pdfminer.six
!pip install unstructured_inference
!pip install pikepdf
!pip install pydf
!pip install sentence-transformers
!pip install chromadb
!pip install unstructured_pytesseract
!pip install python-dotenv
!pip install huggingface_hub
!pip install pillow_heif
!pip install pypdf
!pip install huggingface_hub
!pip install faiss-gpu
!pip install sentence-transformers
!pip install gradio
!pip install openai
!pip install python-dotenv
!pip install tiktoken

In [ ]:
from langchain.document_loaders import OnlinePDFLoader

In [ ]:
parties_report_url_paper = 'http://www.kumoro.staff.ugm.ac.id/file_artikel/Full%20paper-Serving%20the%20Political%20Parties.pdf'
document_loader = OnlinePDFLoader(parties_report_url_paper)

In [ ]:
doc_data = document_loader.load()

In [ ]:
doc_data

### Task 2 : Process Document

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
text_splits = text_splitter.split_documents(doc_data)

In [ ]:
for text in range(len(text_splits)):
  text_splits[text].page_content = text_splits[text].page_content.replace("\n", " ")

In [ ]:
len(text_splits)

In [ ]:
print(text_splits[1].page_content)

### Task 3 : Adding Embedding Model

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
dir(embedding_model)

In [ ]:
# List of question

q1 = 'What significant historical events have shaped the political landscape in Indonesia, particularly regarding decentralization and democratization?'
q2 = 'How do political parties influence the policy-making process at the local level in Indonesia, especially in relation to elections for regional government positions?'
q3 = 'What challenges faced by local governments in Indonesia in terms of ensuring accountability and responsiveness to the needs of their constituents?'
q4 = 'How does the issue of money politics manifest itself in Indonesian elections, particularly in the context of decentralization and democratization?'
q5 = 'What controversies surround the construction of the Musi III bridge in Palembang?'

list_question = [q1, q2, q3, q4, q5]


In [ ]:
# Check the size of embbeded question model

for question in list_question:
  print(f'Size of embbedded model q: {len(embedding_model.embed_query(question))}')

In [ ]:
doc_text = text_splits[5].page_content

In [ ]:
embeddings_dict = {}

for i, question in enumerate(list_question, 1):
    query_vec = embedding_model.embed_query(question)
    embeddings_dict[f'query_vec_q{i}'] = query_vec

doc_vec = embedding_model.embed_query(doc_text)

In [ ]:
embeddings_dict

In [ ]:
len(doc_vec) == len(query_vec)

### Task 4 : Adding Vector Store

In [ ]:
from langchain_community.vectorstores import Chroma

In [ ]:
vector_db = Chroma.from_documents(documents=text_splits,
                                    embedding=embedding_model)

In [ ]:
help(vector_db.similarity_search)

In [ ]:
dir(vector_db)

In [ ]:
results = {}

for i, question in enumerate(list_question, 1):
    results[f'result_q{i}'] = vector_db.similarity_search(query=question, k=10)

result_with_score = {}

for i, question in enumerate(list_question, 1):
    result_with_score[f'result_q{i}'] = vector_db.similarity_search_with_score(query=question, k=10)

In [ ]:
result_with_score['result_q1']

In [ ]:
results['result_q1'][0].page_content

In [ ]:
results['result_q1'][-1].page_content

### Task 5 : Create Retriever

In [ ]:
retriever = vector_db.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 10})

In [ ]:
# to perform similarity search based on query we can run
retrieved_docs = {}

for i, question in enumerate(list_question, 1):
    retrieved_docs[f'result_q{i}'] = retriever.invoke(question)

In [ ]:
len(retrieved_docs['result_q1'])

In [ ]:
print(retrieved_docs['result_q1'][0].page_content)

In [ ]:
print(retrieved_docs['result_q1'][2].page_content)

### Task 6 : Adding Language Model as Generator

In [ ]:
import os
from google.colab import userdata

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('TOKEN')

In [ ]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    model_kwargs={'max_token':1000},
    task="text-generation")

In [ ]:
help(llm.generate)

In [ ]:
print(llm.generate(prompts=['mention the highest mountain in the world']).generations[0][0].text)

### Task 7 : Query

In [ ]:
from langchain_core.prompts import PromptTemplate

"""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.
"""

template = """

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
# Example input model
input_model = custom_rag_prompt.format(context='Cristiano Ronaldo is player of Barcelona',question='Where is Cristiano Ronaldo play football ?')

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
llm_invoke = {}

for i, question in enumerate(list_question, 1):
    llm_invoke[f'result_q{i}'] = llm.invoke(question)

In [ ]:
llm_invoke

In [ ]:
docs = {}

for i, question in enumerate(list_question, 1):
    docs[f'result_q{i}'] = retriever.invoke(question)

In [ ]:
formated_q1 = format_docs(docs['result_q1'])
formated_q2 = format_docs(docs['result_q2'])
formated_q3 = format_docs(docs['result_q3'])
formated_q4 = format_docs(docs['result_q4'])
formated_q5 = format_docs(docs['result_q5'])

In [ ]:
input_model_q1 = custom_rag_prompt.format(context=formated_q1,question=q1)
input_model_q2 = custom_rag_prompt.format(context=formated_q2,question=q2)
input_model_q3 = custom_rag_prompt.format(context=formated_q3,question=q3)
input_model_q4 = custom_rag_prompt.format(context=formated_q4,question=q4)
input_model_q5 = custom_rag_prompt.format(context=formated_q5,question=q5)

In [ ]:
llm.generate(prompts=[input_model_q1])

In [ ]:
llm.generate(prompts=[input_model_q2])

In [ ]:
llm.generate(prompts=[input_model_q3])

In [ ]:
llm.generate(prompts=[input_model_q4])

In [ ]:
llm.generate(prompts=[input_model_q5])

In [ ]:
print(qa_chain.invoke(q1))

In [ ]:
print(qa_chain.invoke(q2))

In [ ]:
print(qa_chain.invoke(q3))

In [ ]:
print(qa_chain.invoke(q4))

In [ ]:
print(qa_chain.invoke(q5))

In [ ]:
print(llm.generate(prompts=[q1]))
print(llm.generate(prompts=[q2]))
print(llm.generate(prompts=[q3]))
print(llm.generate(prompts=[q4]))
print(llm.generate(prompts=[q5]))

### Using FAISS

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS

#### Split the Document

In [ ]:
chunk_size =10
chunk_overlap = 2

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
text_splits = r_splitter.split_text(doc_data[0].page_content)
print(text_splits)

In [ ]:
c_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size=100,
    chunk_overlap=10
)

In [ ]:
charSplit = c_splitter.split_text(doc_data[0].page_content)

print(charSplit)

#### Vector Store in LangChain

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ['OPENAI_API_KEY_2']

openai.api_key  = os.environ['OPENAI_API_KEY_2']
#openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 25
)

In [ ]:
splits = text_splitter.split_text(doc_data[0].page_content)

In [ ]:
type(doc_data[0].page_content)

In [ ]:
len(splits)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
db = FAISS.from_documents(doc_data, hf)

#### Querying Data

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import HuggingFaceHub
from getpass import getpass

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
repo_id = "google/flan-t5-xxl"

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5}
)

qa = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

#### Deployment

In [ ]:
import gradio as gr

In [ ]:
#write your Gradio implementation here


def chat_bot(Textbox : gr.components.Textbox) -> str:
    return qa.run(Textbox)

iface = gr.Interface(fn=chat_bot, inputs="text", outputs="text")
iface.launch()